In [1]:
import configparser
from datetime import datetime
import os
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf, col
from pyspark.sql.functions import year, month, dayofmonth, hour, weekofyear, date_format, dayofweek, monotonically_increasing_id
from pyspark.sql.types import TimestampType

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
1,application_1665862418527_0002,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
input_data = "s3a://udacity-dend/"
output_data = "s3a://udacity-data-lake-project-four/"

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
# get filepath to song data file
song_data = input_data + 'song_data/A/A/A/'

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
# read song data file
df = spark.read.json(song_data)
df.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- artist_id: string (nullable = true)
 |-- artist_latitude: double (nullable = true)
 |-- artist_location: string (nullable = true)
 |-- artist_longitude: double (nullable = true)
 |-- artist_name: string (nullable = true)
 |-- duration: double (nullable = true)
 |-- num_songs: long (nullable = true)
 |-- song_id: string (nullable = true)
 |-- title: string (nullable = true)
 |-- year: long (nullable = true)

In [5]:
# extract columns to create songs table
songs_table = df.select(["song_id", "title", "artist_id", "year", "duration"]).filter(col("song_id").isNotNull()).distinct()   

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
# test song data
print("songs_table ", songs_table.count())
songs_table.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

songs_table  24
+------------------+--------------------+------------------+----+---------+
|           song_id|               title|         artist_id|year| duration|
+------------------+--------------------+------------------+----+---------+
|SODZYPO12A8C13A91E|Burn My Body (Alb...|AR1C2IX1187B99BF74|   0|177.99791|
|SOIGHOD12A8C13B5A1|        Indian Angel|ARY589G1187B9A9F4E|2004|171.57179|
|SOOVHYF12A8C134892|     I'll Be Waiting|ARCLYBR1187FB53913|1989|304.56118|
|SOAPERH12A58A787DC|The One And Only ...|ARZ5H0P1187B98A1DD|   0|230.42567|
|SOHKNRJ12A6701D1F8|        Drop of Rain|AR10USD1187B99F3F1|   0|189.57016|
|SOHOZBI12A8C132E3C|         Smash It Up|AR0MWD61187B9B2B12|2000|195.39546|
|SOERIDA12A6D4F8506|I Want You (Album...|ARBZIN01187FB362CC|2006|192.28689|
|SOXZYWX12A6310ED0C|     It's About Time|ARC1IHZ1187FB4E920|   0| 246.9873|
|SOBLFFE12AF72AA5BA|              Scream|ARJNIUY12298900C91|2009| 213.9424|
|SOTAZDY12AB0187616|            Drillbit|ARZKCQM1257509D107|   0|374.621

In [7]:
# write songs table to parquet files partitioned by year and artist
songs_table.write.mode('overwrite').parquet(output_data + "song-table-data/songs.parquet", partitionBy=['year','artist_id'])

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [8]:
#test song parquet
testparquet = spark.read.parquet(output_data + "song-table-data/songs.parquet")
testparquet.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- song_id: string (nullable = true)
 |-- title: string (nullable = true)
 |-- duration: double (nullable = true)
 |-- year: integer (nullable = true)
 |-- artist_id: string (nullable = true)

In [9]:
 # extract columns to create artists table
artists_table = df.select(["artist_id", "artist_name", "artist_location", "artist_latitude", "artist_longitude"]).filter(col("artist_id").isNotNull()).distinct()


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [10]:
print("artists_table", artists_table.count())
artists_table.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

artists_table 24
+------------------+--------------------+--------------------+---------------+----------------+
|         artist_id|         artist_name|     artist_location|artist_latitude|artist_longitude|
+------------------+--------------------+--------------------+---------------+----------------+
|ARC1IHZ1187FB4E920|        Jamie Cullum|                    |           null|            null|
|ARZKCQM1257509D107|          Dataphiles|                    |           null|            null|
|AREWD471187FB49873|            Son Kite|                    |           null|            null|
|ARGE7G11187FB37E05|        Cyndi Lauper|        Brooklyn, NY|           null|            null|
|ARSVTNL1187B992A91|       Jonathan King|     London, England|       51.50632|        -0.12714|
|AR9Q9YC1187FB5609B|    Quest_ Pup_ Kevo|          New Jersey|           null|            null|
|AR73AIO1187B9AD57B|   Western Addiction|   San Francisco, CA|       37.77916|      -122.42005|
|AR0MWD61187B9B2B12|Int

In [11]:
# write artists table to parquet files
artists_table.write.mode('overwrite').parquet(output_data + "artist-table-data/artists.parquet", partitionBy=['artist_id'])
   

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [12]:
# test artist parquet
testparquet = spark.read.parquet(output_data + "artist-table-data/artists.parquet")
testparquet.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- artist_name: string (nullable = true)
 |-- artist_location: string (nullable = true)
 |-- artist_latitude: double (nullable = true)
 |-- artist_longitude: double (nullable = true)
 |-- artist_id: string (nullable = true)

In [13]:
# get filepath to log data file
log_data = input_data + 'log_data/2018/11/2018-11-01-events.json'

# read log data file
df = spark.read.json(log_data)
df.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- artist: string (nullable = true)
 |-- auth: string (nullable = true)
 |-- firstName: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- itemInSession: long (nullable = true)
 |-- lastName: string (nullable = true)
 |-- length: double (nullable = true)
 |-- level: string (nullable = true)
 |-- location: string (nullable = true)
 |-- method: string (nullable = true)
 |-- page: string (nullable = true)
 |-- registration: double (nullable = true)
 |-- sessionId: long (nullable = true)
 |-- song: string (nullable = true)
 |-- status: long (nullable = true)
 |-- ts: long (nullable = true)
 |-- userAgent: string (nullable = true)
 |-- userId: string (nullable = true)

In [14]:
# filter by actions for song plays
df = df.filter('page = "NextSong"') 

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [15]:
# extract columns for users table     
users_table = df.select([col("userId").alias("user_id"),\
                         col("firstName").alias("first_name"),\
                         col("lastName").alias("last_name"),\
                         "gender", "level"]).filter(col("userId").isNotNull()).distinct()

print("users_table", users_table.count())
users_table.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

users_table 4
+-------+----------+---------+------+-----+
|user_id|first_name|last_name|gender|level|
+-------+----------+---------+------+-----+
|     26|      Ryan|    Smith|     M| free|
|    101|    Jayden|      Fox|     M| free|
|     10|    Sylvie|     Cruz|     F| free|
|      8|    Kaylee|  Summers|     F| free|
+-------+----------+---------+------+-----+

In [16]:
# write users table to parquet files
users_table.write.mode('overwrite').parquet(output_data + "user-table-data/users.parquet", partitionBy=['user_id'])

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [17]:
# test users parquet
testparquet = spark.read.parquet(output_data + "user-table-data/users.parquet")
testparquet.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- first_name: string (nullable = true)
 |-- last_name: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- level: string (nullable = true)
 |-- user_id: integer (nullable = true)

In [18]:
# create timestamp column from original timestamp column
get_timestamp = udf(lambda x: datetime.fromtimestamp(x / 1000.0), TimestampType())
df = df.withColumn('timestamp', get_timestamp(col('ts')))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [19]:
# extract columns to create time table
time_table = df.select("timestamp").alias("start_time")\
                        .withColumn("hour", hour(col('timestamp')))\
                        .withColumn("day", dayofmonth(col('timestamp')))\
                        .withColumn("week", weekofyear(col('timestamp')))\
                        .withColumn("month", month(col('timestamp')))\
                        .withColumn("year", year(col('timestamp')))\
                        .withColumn("weekday", dayofweek(col('timestamp')))\

print("time_table", time_table.count())
time_table.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

time_table 11
+--------------------+----+---+----+-----+----+-------+
|           timestamp|hour|day|week|month|year|weekday|
+--------------------+----+---+----+-----+----+-------+
|2018-11-01 21:01:...|  21|  1|  44|   11|2018|      5|
|2018-11-01 21:05:...|  21|  1|  44|   11|2018|      5|
|2018-11-01 21:08:...|  21|  1|  44|   11|2018|      5|
|2018-11-01 21:11:...|  21|  1|  44|   11|2018|      5|
|2018-11-01 21:17:...|  21|  1|  44|   11|2018|      5|
|2018-11-01 21:24:...|  21|  1|  44|   11|2018|      5|
|2018-11-01 21:28:...|  21|  1|  44|   11|2018|      5|
|2018-11-01 21:42:...|  21|  1|  44|   11|2018|      5|
|2018-11-01 21:52:...|  21|  1|  44|   11|2018|      5|
|2018-11-01 21:55:...|  21|  1|  44|   11|2018|      5|
|2018-11-01 22:23:...|  22|  1|  44|   11|2018|      5|
+--------------------+----+---+----+-----+----+-------+

In [20]:
# write time table to parquet files partitioned by year and month
time_table.write.mode('overwrite').parquet(output_data + "time-table-data/time.parquet", partitionBy=['year', 'month'])

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

----------------------------------------
Exception happened during processing of request from ('127.0.0.1', 53316)
Traceback (most recent call last):
  File "/usr/lib64/python3.6/socketserver.py", line 320, in _handle_request_noblock
    self.process_request(request, client_address)
  File "/usr/lib64/python3.6/socketserver.py", line 351, in process_request
    self.finish_request(request, client_address)
  File "/usr/lib64/python3.6/socketserver.py", line 364, in finish_request
    self.RequestHandlerClass(request, client_address, self)
  File "/usr/lib64/python3.6/socketserver.py", line 724, in __init__
    self.handle()
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/accumulators.py", line 266, in handle
    poll(authenticate_and_accum_updates)
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/accumulators.py", line 241, in poll
    if func():
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/accumulators.py", line 254, in authenticate_and_accum_updates
    received_to

In [21]:
# test time parquet
testparquet = spark.read.parquet(output_data + "time-table-data/time.parquet")
testparquet.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- timestamp: timestamp (nullable = true)
 |-- hour: integer (nullable = true)
 |-- day: integer (nullable = true)
 |-- week: integer (nullable = true)
 |-- weekday: integer (nullable = true)
 |-- year: integer (nullable = true)
 |-- month: integer (nullable = true)

In [22]:
# read song data file
song_df = spark.read.json(song_data)
song_df.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- artist_id: string (nullable = true)
 |-- artist_latitude: double (nullable = true)
 |-- artist_location: string (nullable = true)
 |-- artist_longitude: double (nullable = true)
 |-- artist_name: string (nullable = true)
 |-- duration: double (nullable = true)
 |-- num_songs: long (nullable = true)
 |-- song_id: string (nullable = true)
 |-- title: string (nullable = true)
 |-- year: long (nullable = true)

In [23]:
# extract columns from joined song and log datasets to create songplays table 
cond = [song_df.title == df.song, song_df.artist_name == df.artist]
songplays_table = df.join(song_df, cond, "left")
songplays_table = songplays_table.select([monotonically_increasing_id().alias("songplay_id"),\
                                          col("timestamp").alias("start_time"),\
                                          col("userId").alias("user_id"),\
                                          "level",\
                                          "song_id",\
                                          "artist_id",\
                                          col("sessionId").alias("session_id"),\
                                          "location",\
                                          col("userAgent").alias("user_agent"),\
                                          year(col("timestamp")).alias("year"),\
                                          month(col("timestamp")).alias("month")]).filter(col("song").isNotNull()).distinct()

print("songplays_table", songplays_table.count())
songplays_table.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

songplays_table 11
+-----------+--------------------+-------+-----+-------+---------+----------+--------------------+--------------------+----+-----+
|songplay_id|          start_time|user_id|level|song_id|artist_id|session_id|            location|          user_agent|year|month|
+-----------+--------------------+-------+-----+-------+---------+----------+--------------------+--------------------+----+-----+
|          3|2018-11-01 21:11:...|      8| free|   null|     null|       139|Phoenix-Mesa-Scot...|"Mozilla/5.0 (Win...|2018|   11|
|          7|2018-11-01 21:42:...|     10| free|   null|     null|         9|Washington-Arling...|"Mozilla/5.0 (Mac...|2018|   11|
|          5|2018-11-01 21:24:...|      8| free|   null|     null|       139|Phoenix-Mesa-Scot...|"Mozilla/5.0 (Win...|2018|   11|
|          8|2018-11-01 21:52:...|     26| free|   null|     null|       169|San Jose-Sunnyval...|"Mozilla/5.0 (X11...|2018|   11|
|          1|2018-11-01 21:05:...|      8| free|   null|     nul

In [24]:
# write songplays table to parquet files partitioned by year and month
songplays_table.write.mode('overwrite').parquet(output_data + "songplays-table-data/songplays.parquet", partitionBy=["year", "month"])

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [25]:
# test time parquet
testparquet = spark.read.parquet(output_data + "songplays-table-data/songplays.parquet")
testparquet.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- songplay_id: long (nullable = true)
 |-- start_time: timestamp (nullable = true)
 |-- user_id: string (nullable = true)
 |-- level: string (nullable = true)
 |-- song_id: string (nullable = true)
 |-- artist_id: string (nullable = true)
 |-- session_id: long (nullable = true)
 |-- location: string (nullable = true)
 |-- user_agent: string (nullable = true)
 |-- year: integer (nullable = true)
 |-- month: integer (nullable = true)